In [1]:
#r "nuget:Microsoft.ML,1.5.5"
#r "nuget:Microsoft.ML.TimeSeries,1.5.5"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML.TimeSeries version 1.5.5

Installed package Microsoft.ML version 1.5.5

In [2]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms.TimeSeries;

## Model Input & Outpot Schema

The `ModelInput` and `ModelOutput` classes define the data schema the model is trained on as well as the output generated by the model.

In [3]:
public class ModelInput
{
    [LoadColumn(6)]
    public DateTime Date { get; set; }

    [LoadColumn(7)]
    public float MaxTemp { get; set; }
}

public class ModelOutput
{

    public float[] ForecastTemp { get; set; }

    public float[] LowerBoundTemp { get; set; }

    public float[] UpperBoundTemp { get; set; }
}

## Train

### Initialize MLContext

In [4]:
MLContext mlContext = new MLContext();

### Load the data

In [5]:
var dataPath = "Data/seattle-10yr.csv";
var dataView = mlContext.Data.LoadFromTextFile<ModelInput>(dataPath, hasHeader:true, separatorChar:',');


### Split into train and test sets

In [6]:
var data = mlContext.Data.CreateEnumerable<ModelInput>(dataView, reuseRowObject: false);
var rowCount = data.Count();
var split = (int) (rowCount / 2f);

In [7]:
IDataView trainingDataView = mlContext.Data.LoadFromEnumerable(data.Take(split));
IDataView testDataView = mlContext.Data.LoadFromEnumerable(data.TakeLast(split-1));

### Build model

In [8]:
var maxTempEstimator = mlContext.Forecasting.ForecastBySsa(
    outputColumnName: "ForecastTemp",
    inputColumnName: "MaxTemp",
    windowSize: 5,
    seriesLength: 30,
    trainSize: 4027,
    horizon: 5,
    confidenceLevel: 0.90f,
    confidenceLowerBoundColumn: "LowerBoundTemp",
    confidenceUpperBoundColumn: "UpperBoundTemp");

In [9]:
ITransformer maxTempModel = maxTempEstimator.Fit(trainingDataView);

## Evaluate the model

In [10]:
IDataView predictions = maxTempModel.Transform(testDataView);

In [11]:
var actual = 
    mlContext.Data.CreateEnumerable<ModelInput>(testDataView, reuseRowObject: true)
        .Select(observed => new {Date= observed.Date, ActualTemp= observed.MaxTemp});

var forecast = 
    mlContext.Data.CreateEnumerable<ModelOutput>(predictions, reuseRowObject: true)
        .Select(forecast => new {ForecastTemp= forecast.ForecastTemp[0], UpperBound=forecast.UpperBoundTemp[0], LowerBound = forecast.LowerBoundTemp[0]});

In [12]:
var combined = actual.Zip(forecast,(actualValue,forecastValue) => 
    new 
    {
        Date=actualValue.Date,
        ActualTemp=actualValue.ActualTemp,
        ForecastTemp=forecastValue.ForecastTemp,
        UpperBound=forecastValue.UpperBound,
        LowerBound=forecastValue.LowerBound
    });

In [13]:
display(combined);

index,Date,ActualTemp,ForecastTemp,UpperBound,LowerBound
0,2015-10-08 00:00:00Z,71,69.40734,75.97925,62.835434
1,2015-10-09 00:00:00Z,70,73.34907,79.920975,66.77716
2,2015-10-10 00:00:00Z,72,72.568665,79.14057,65.99676
3,2015-10-11 00:00:00Z,67,65.23682,71.80872,58.66491
4,2015-10-12 00:00:00Z,68,67.14516,73.717064,60.57325
5,2015-10-13 00:00:00Z,63,64.414,70.98591,57.842094
6,2015-10-14 00:00:00Z,63,64.47168,71.04359,57.899773
7,2015-10-15 00:00:00Z,68,69.88303,76.45494,63.311127
8,2015-10-16 00:00:00Z,70,72.80112,79.373024,66.22921
9,2015-10-17 00:00:00Z,68,65.56419,72.1361,58.992287


In [14]:
var metrics = combined.Select(data => data.ActualTemp - data.ForecastTemp);

In [15]:
var MAE = metrics.Average(error => Math.Abs(error)); // Mean Absolute Error
var RMSE = Math.Sqrt(metrics.Average(error => Math.Pow(error, 2))); // Root Mean Squared Error

In [16]:
Console.WriteLine("Evaluation Metrics");
Console.WriteLine("---------------------");
Console.WriteLine($"Mean Absolute Error: {MAE:F3}");
Console.WriteLine($"Root Mean Squared Error: {RMSE:F3}\n");    

Evaluation Metrics
---------------------
Mean Absolute Error: 3.078
Root Mean Squared Error: 5.213



## Retrain model with all the data

In [17]:
ITransformer maxTempModel = maxTempEstimator.Fit(dataView);

## Save the model

In [18]:
var maxForecastEngine = maxTempModel.CreateTimeSeriesEngine<ModelInput, ModelOutput>(mlContext);

In [19]:
maxForecastEngine.CheckPoint(mlContext,"WeatherForecastModel.zip");